# Tool

Functions and services an LLM can utilize to extend its capabilities.

Tools are interfaces that an agent can use to interact with the world. They combine a few things:

- The name of the tool
- A description of what the tool is
- JSON schema of what the inputs to the tool are
- The function to call
- Whether the result of a tool should be returned directly to the user

[Langchain Docs](https://python.langchain.com/docs/modules/agents/tools/)

In [1]:
import os
import dotenv
import openai

dotenv_path = dotenv.find_dotenv()
dotenv.load_dotenv(dotenv_path)

True

In [2]:
# tool decorator
from langchain.agents import tool


In [3]:
import requests
from pydantic import BaseModel, Field
import datetime

# Define the input schema
class OpenMeteoInput(BaseModel):
    latitude: float = Field(..., description="Latitude of the location to fetch weather data for")
    longitude: float = Field(..., description="Longitude of the location to fetch weather data for")

@tool(args_schema=OpenMeteoInput)
def get_current_temperature(latitude: float, longitude: float) -> dict:
    """Fetch current temperature for given coordinates."""
    
    BASE_URL = "https://api.open-meteo.com/v1/forecast"
    
    # Parameters for the request
    params = {
        'latitude': latitude,
        'longitude': longitude,
        'hourly': 'temperature_2m',
        'forecast_days': 1,
    }

    # Make the request
    response = requests.get(BASE_URL, params=params)
    
    if response.status_code == 200:
        results = response.json()
    else:
        raise Exception(f"API Request failed with status code: {response.status_code}")

    current_utc_time = datetime.datetime.utcnow()
    time_list = [datetime.datetime.fromisoformat(time_str.replace('Z', '+00:00')) for time_str in results['hourly']['time']]
    temperature_list = results['hourly']['temperature_2m']
    
    closest_time_index = min(range(len(time_list)), key=lambda i: abs(time_list[i] - current_utc_time))
    current_temperature = temperature_list[closest_time_index]
    
    return f'The current temperature is {current_temperature}°C'

ValidationError: 1 validation error for StructuredTool
args_schema
  subclass of BaseModel expected (type=type_error.subclass; expected_class=BaseModel)

In [3]:
# converts the function into a LangChain tool that can be used in the pipeline.
@tool
def search(query: str) -> str:
    """Search for weather online"""
    return "42f"

In [4]:
search.name
search.description


'search(query: str) -> str - Search for weather online'

In [5]:
search.args

{'query': {'title': 'Query', 'type': 'string'}}

In [6]:
from pydantic import BaseModel, Field
class SearchInput(BaseModel):
    query: str = Field(description="Thing to search for")


In [12]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [13]:
api_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=100)
tool = WikipediaQueryRun(api_wrapper=api_wrapper)

In [14]:
tool.name

'wikipedia'

In [15]:
tool.args

{'query': {'title': 'Query', 'type': 'string'}}

In [16]:
tool.run({"query": "langchain"})

'Page: LangChain\nSummary: LangChain is a framework designed to simplify the creation of applications '

In [17]:
from langchain_core.pydantic_v1 import BaseModel, Field


class WikiInputs(BaseModel):
    """Inputs to the wikipedia tool."""

    query: str = Field(
        description="query to look up in Wikipedia, should be 3 or less words"
    )

In [18]:
tool = WikipediaQueryRun(
    name="wiki-tool",
    description="look up things in wikipedia",
    args_schema=WikiInputs,
    api_wrapper=api_wrapper,
    return_direct=True,
)

In [19]:
tool.run("langchain")

'Page: LangChain\nSummary: LangChain is a framework designed to simplify the creation of applications '

In [9]:
from langchain_core.tools import tool


@tool
def multiply(first_int: int, second_int: int, third_int: int) -> int:
    """Multiply two integers together."""
    return first_int * second_int * third_int

In [10]:
multiply.invoke({"first_int": 4, "second_int": 5, "third_int": 5})

100